In [92]:
# libs

import pandas as pd
import numpy as np

import re
import requests
from bs4 import BeautifulSoup
import json
import dpath.util

import time
import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

In [84]:
time.time()

AttributeError: 'float' object has no attribute 'time'

In [89]:
def print_worktime(start):
    
    duration = time.time() - start
    duration = round(duration / 60, 3)
    
    print(f"Код отработал за: {duration} минут")

# Поиск пути к блоку с изображениями

In [17]:
url = 'https://www.gapminder.org/dollar-street?zoom=6&p=4'
text = requests.get(url)
content = BeautifulSoup(text.text, "html.parser")

In [18]:
print(content.prettify())

<!DOCTYPE html>
<html>
 <head>
  <script src="/cdn-cgi/apps/head/BA9wG71UcNHRZyqda6gYchEbuNM.js">
  </script>
  <link href="/dollar-street/static/favicons/favicon.ico" rel="icon" type="image/x-icon"/>
  <link href="/dollar-street/static/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/dollar-street/static/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/dollar-street/static/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/dollar-street/static/favicons/site.webmanifest" rel="manifest"/>
  <link color="#ffcc55" href="/dollar-street/static/favicons/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="#ffcc55" name="msapplication-TileColor"/>
  <meta content="/dollar-street/static/favicons/mstile-144x144.png" name="msapplication-TileImage"/>
  <meta content="#ffffff" name="theme-color"/>
  <meta content="width=device-width" name="viewport"/>
  <meta charset="utf-8"/>
  <me

##### Блок с нужной информацией и изображениями находится по тегу **script** с аттрибутом **id="__NEXT_DATA__"**

##### Далее извлекаем данные из блока

In [19]:
json_data = content.find_all('script',id="__NEXT_DATA__")
json_data = json_data[0].contents[0]
json_data = json.loads(json_data)

In [20]:
json_data.keys()

dict_keys(['props', 'page', 'query', 'buildId', 'assetPrefix', 'isFallback', 'customServer', 'gip', 'appGip'])

In [21]:
print(dpath.util.search(json_data, '**/original'))

<ipython-input-21-3d477a9780b5>:1: DeprecationWarning: The dpath.util package is being deprecated. All util functions have been moved to dpath package top level.
  print(dpath.util.search(json_data, '**/original'))


{'props': {'pageProps': {'initialMobxState': {'thingStore': {'mediaRegistry': [{'images': {'original': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30.jpg'}}, {'images': {'original': 'https://media.dollarstreet.org/5ec4f87df0611d7ddd740f47/5ec4f87df0611d7ddd740f47.jpg'}}, {'images': {'original': 'https://media.dollarstreet.org/5ec4f7d3f0611d7ddd740977/5ec4f7d3f0611d7ddd740977.JPG'}}, {'images': {'original': 'https://media.dollarstreet.org/5d4be719cf0b3a0f3f344e46/5d4be719cf0b3a0f3f344e46.jpg'}}, {'images': {'original': 'https://media.dollarstreet.org/5d4be102cf0b3a0f3f33a493/5d4be102cf0b3a0f3f33a493.jpeg'}}, {'images': {'original': 'https://media.dollarstreet.org/5d4be139cf0b3a0f3f33aaf3/5d4be139cf0b3a0f3f33aaf3.jpg'}}, {'images': {'original': 'https://media.dollarstreet.org/5d4bf210cf0b3a0f3f3579ec/5d4bf210cf0b3a0f3f3579ec.jpg'}}, {'images': {'original': 'https://media.dollarstreet.org/62a667b19280e31f48f9d92b/905cc428e0f046d9095b61b030d4b100/62a667b1

**Путь к изображениям**

props &rarr; pageProps &rarr; initialMobxState &rarr; thingStore &rarr; mediaRegistry &rarr; images

In [22]:
print('Всего изображений на 4 странице:', len(json_data['props']['pageProps']['initialMobxState']['thingStore']['mediaRegistry']))

Всего изображений на 4 странице: 60


In [27]:
data_page = json_data['props']['pageProps']['initialMobxState']['thingStore']['mediaRegistry']

In [41]:
data_page[0]

{'id': '5d4bf04ccf0b3a0f3f354b30',
 'images': {'original': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30.jpg',
  'uncropped': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30_raw.jpg',
  'cropped80': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30_80.jpg',
  'cropped180': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30_180.jpg',
  'cropped360': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30_360.jpg',
  'cropped640': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30_640.jpg',
  'full512': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30_512.jpg',
  'full1024': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30_1024.jpg',
  'full2048': 'https://media.dollarstreet.org/5d4bf04ccf0b3a0f3f354b30/5d4bf04ccf0b3a0f3f354b30_2048.jpg

# Функция парсера

In [64]:
def parser(page: int, topic: str, country: str = 'world', media: str = 'photos') -> pd.DataFrame:
    
    url_body = 'https://www.gapminder.org/dollar-street?'
    p = 'p=' + str(page)
    media = 'media=' + media
    topic_url = 'topic=' + topic
    country = 'country=' + country
    url = '&'.join((url_body, topic_url, country, media, p))
    
    text = requests.get(url)
    content = BeautifulSoup(text.text, "html.parser")
    json_data = content.find_all('script',id="__NEXT_DATA__")
    json_data = json_data[0].contents[0]
    json_data = json.loads(json_data)
    data_page = json_data['props']['pageProps']['initialMobxState']['thingStore']['mediaRegistry']
    
    id_code = []
    images = []
    countries = []
    regions = []
    incomes = []
    
    for item in range(len(data_page)):
        images.append(data_page[item]['images']['original'])
        countries.append(data_page[item]['country']['name'])
        regions.append(data_page[item]['region']['name'])
        incomes.append(data_page[item]['sortVal'])
        id_code.append(data_page[item]['id'])
        
    df = pd.DataFrame(columns = ['id', 'image', 'country', 'region', 'income'])
    df['id'] = id_code
    df['image'] = images
    df['country'] = countries
    df['region'] = regions
    df['income'] = incomes
    df['page'] = page
    df['topic'] = topic
    
    return df

In [65]:
first_page = parser(1, 'families')
first_page.head(2)

,id,image,country,region,income,page,topic
0,5d4bf10dcf0b3a0f3f355e40,https://media.dollarstreet.org/5d4bf10dcf0b3a0...,Cote d'Ivoire,Africa,41.767077,1,families
1,5d4be794cf0b3a0f3f345b34,https://media.dollarstreet.org/5d4be794cf0b3a0...,Bangladesh,Asia,175.000000,1,families


In [67]:
first_page.shape

(60, 7)

# Парсинг всех страниц и тем

In [71]:
pages = np.arange(1,11)
topics = ['families', 'adding-spices-to-food-while-cooking', 'armchairs', 'backyards', 'bathroom-doors', 'bathrooms', 
          'bedrooms', 'beds', 'bikes', 'books', 'bowls', 'ceilings', 'chikens', 'child-rooms', 'chopping-food',
         'cleaning-equipment', 'cooking', 'cooking-pots', 'cooking-utensils', 'cups', 'dish-brushes', 'dish-racks',
         'dish-washing-soaps', 'drying-clothes', 'eating', 'everyday-shoes', 'floors', 'front-doors', 'get-water',
         'guest-beds', 'hair-brushes', 'hallways', 'hand-washing', 'hands', 'homes', 'jackets', 'kids-bed', 'kitchen-sinks',
         'kitchens', 'light-sources-in-livingroom', 'light-sources-kitchen', 'lock-on-front-doors', 'make-up', 'meat', 
          'medication', 'mosquito-protections', 'nicest-shoes', 'palms', 'pets', 'phones', 'places-for-dinner', 
          'places-where-guests-are-served-dinner', 'plates', 'plates-of-food', 'power-outlets', 'radios', 'roofs',
         'salt', 'shampoo', 'showers', 'sitting-areas', 'soaps', 'sofas', 'sources-of-cool', 'spices', 'storage-rooms',
         'stoves', 'street-details', 'street-view', 'tables-with-food', 'teeth', 'toilets', 'tools', 'tooth-paste',
         'toothbrushes', 'toys', 'trash', 'tvs', 'wall-clocks', 'wall-decorations', 'walls-inside', 'wardrobes',
         'washing-clothes', 'washing-detergent', 'washing-hands', 'water-outlets', 'wheel-barrows', 'work-areas']

In [101]:
start = time.time()

df = pd.DataFrame(columns=first_page.columns)

for t,s in zip(topics, tqdm.notebook.tqdm(range(len(topics)-1))):
    for p in pages:
        
        page = parser(page=p, topic=t)
        df = pd.concat([df, page])
        
print_worktime(start)


Код отработал за: 23.641 минут


In [103]:
df.to_excel('parsered_ds.xlsx')